## FINANCIAL DATA

MODULE 2 | LESSON 2


---


# IDENTIFYING AND APPLYING RISK METRICS ASSOCIATED WITH FINANCIAL MARKETS

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** | Stock indices, Log return calculations  |
|**Keywords** | Variance, Volatility, Standard Deviation, Moving Averages |

---


*In the previous lesson, we examined the returns of different asset classes and performed some basic price analysis. In this lesson, we will expand on what we learned in the previous lesson with a few custom volatility metrics, along with how to program them in Python. We'll also focus on the risk, or volatility of returns, of financial assets.*

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web

pd.options.display.float_format = "{:,.3f}".format
import datetime
from datetime import date

import seaborn as sns
from IPython.display import VimeoVideo
from matplotlib import pyplot as plt

## 1. Future Value of an Investment
The S&P 500 consists of 500 large-cap companies. The index calculates weights according to market capitalization. This weighting is different from that in the Dow Jones Industrial Average, which is price weighted. The companies that make up this index are chosen by a committee, but companies must fit specific criteria before being included (Standard & Poor's):

* Market capitalization must be greater than or equal to \\$13.1 billion.
* Annual dollar value traded to float-adjusted market capitalization is greater than 1.0.
* The minimum monthly trading volume is 250,000 shares in each of the six months leading up to the evaluation date.
* The company must be publicly listed on either the New York Stock Exchange (NYSE), including NYSE Arca or NYSE American, or NASDAQ (NASDAQ Global Select Market, NASDAQ Select Market or the NASDAQ Capital Market).
* The company should be from the U.S.
* Securities that are ineligible for inclusion in the index are limited partnerships, master limited partnerships and their investment trust units, OTC Bulletin Board issues, closed-end funds, exchange-traded funds, exchange-traded notes, royalty trusts, tracking stocks, preferred stock, unit trusts, equity warrants, convertible bonds, investment trusts, American depositary receipts, and American depositary shares.
* Since 2017, companies with dual share classes, like Standard & Poor's, are not added to the index.

This will be used as our proxy for large-cap stocks.

The Russell 2000 is an index that contains 2,000 small-cap companies and is frequently used as a benchmark by small-cap mutual funds for comparison purposes. The Russell 2000 is constructed to provide a barometer for small-cap stocks. It's reconstituted annually to ensure companies from the previous year don't get too large and distort the characteristics of small-caps (Maginn 118).

The Russell 2000 Index will be used as our proxy for small-cap companies.

One way to compare two investment opportunities is to determine the future value of an investment. There are always risks and many assumptions that need to be made when determining future value, especially for a risky investment like a stock. We will begin by using the average daily rate of return of each index to determine expected return. This is a simple way of thinking because it’s impossible to say the next 10 years will be the same as the past 10 years, but it is a starting point when comparing investments. We will delve further into more advanced comparison metrics as the course goes on. 

**The formula for compound interest is:**

$FV = PV (1+i)^n$, where

$FV = \textrm{Future value}$
$PV = \textrm{Present value}$
$i = \textrm{Periodic interest rate}$
$n = \textrm{Number of periods}$


### 1.1 Pull 10 years daily price data for S&P 500 and Russell 2000

We're going to dig deeper into specific equities asset classes and compare them as potential investments. We will start by comparing large cap to small cap stocks using S&P 500 and the Russell 2000 Index. 

Throughout this lesson we will build towards a function here which takes a date range and compares two series of returns.


In [ ]:
# start = datetime.date.today()-datetime.timedelta(365*10)
# end = datetime.date.today()
start = datetime.date(2011, 11, 25)
end = datetime.date(2021, 11, 22)

prices = web.DataReader(["^GSPC", "^RUT"], "yahoo", start, end)["Adj Close"]

# Rename column to make names more intuitive
prices = prices.rename(columns={"^GSPC": "SP500", "^RUT": "Russell2000"})

In [ ]:
prices.describe()

In [ ]:
prices.head()

Once again, we can see these datasets have different starting points so the only way we can do a fair comparison is by comparing returns

### 1.2 Calculate log returns, remove unused columns, and drop nulls

No nulls here for the weekend dates. Since S&P 500 and the Russell 2000 Index are both stock indices, the weekends are automatically removed. 

In [ ]:
df = np.log(prices) - np.log(prices.shift(1))
df = df.iloc[1:, 0:]

In [ ]:
df.head()

### 1.3 Calculating Future Value of each Index

Once we get the daily returns in a DataFrame, df, we can use the mean() method in order to determine daily average returns. We calculate this over the last 10 years to determine our expected daily rate of return for our future value calculation. From here, we will assume a \$1,000 investment for present value and plug these numbers into the interest rate parameter. If we assume a 10-year investment time frame and 252 trading days a year, the periods will be 10*252 = 2520.

In [ ]:
df.mean() * 100

Assuming mean of daily returns is the daily expected return going forward

In [ ]:
(1 + df.mean()) ** (252 * 10) * 1000

With our future value calculations, we’re predicting a \\$1,000 investment in the S&P 500 today would yield a \\$4,053.57 future value and for the Russell 2000 a \\$3,509.06 future value. 

While future value is important when comparing investments, we also need to be concerned about how volatile these investment opportunities are. In the next section, we will discuss three different volatility metrics along with how to program them in Python.

## 2. Investment Opportunities: Volatility
### 2.1 Price Volatility: High-Low
We will be using the prices DataFrame again to show some simple ways to compare the volatility of certain investments. The first method is comparing the high and low of the index. For this, we use the max() and min() methods to get the high and low over the duration of the DataFrame by column. From here, we can subtract these from each other to get an idea of the potential volatility of each investment.

In [ ]:
prices.max()

In [ ]:
prices.min()

In [ ]:
prices.max() - prices.min()

While some may find this useful, we definitely need to improve upon this metric to make it more useful when comparing different investments. How do you think we can do this? Let's try out two ideas.

The first is changing the time frame slightly. Above, we’re looking at the last 10 years of data. Perhaps it will be more useful to analyze more recent data. For this, we will look at the same metric over the last year:

In [ ]:
currYear = prices.loc[
    date.today() - datetime.timedelta(365) : date.today()  # noqa E203
]
currYear.max() - currYear.min()

Even when comparing data over the last 365 days, it seems hard to do a comparison of these two datasets since they have different starting points. One final adjustment to this volatility metric could be to standardize it by dividing by the current price of the index:

In [ ]:
(currYear.max() - currYear.min()) / prices.iloc[-1]

Now that we have the data standardized by the current price, this high/low metric actually shows the Russell 2000 to be the more volatile investment. This is pretty much in line with what you would expect since the S&P 500 is mostly big established companies while the Russell 2000 is filled with small caps. Accordingly, you would expect business to be more turbulent and the stock prices to be more volatile.



### 2.2 Moving Averages 
#### 2.2.1 50-Day Moving Average
We are going to create a volatility metric that compares each day's price to the moving average. The 50- and 200-day averages are commonly used when comparing investments. For our metric, we will use the 50-day average. Here is an example of how we can chart the S&P 500 along with its 50-day moving average:

In [ ]:
prices["SP500 50 day_rolling_avg"] = prices.SP500.rolling(50).mean()

# set figure size
plt.figure(figsize=(12, 5))

# plot a simple time series plot
# using seaborn.lineplot()
sns.lineplot(x="Date", y="SP500", data=prices, label="Daily S&P 500 Prices")


# plot using rolling average
sns.lineplot(x="Date", y="SP500 50 day_rolling_avg", data=prices, label="Rollingavg")

#### 2.2.2 50-Day Rolling Distance 
We can create a volatility metric here using this moving average. Let’s imagine we were taking the distance between the moving average line and each data point from the S&P 500 chart. We can use this as a proxy for volatility. We also would want to treat the negative differences the same as the positive differences; otherwise, they may cancel each other out. As such, we will use the absolute value of each point. We will divide these differences by the prices in order to standardize these values, and finally, we'll take the average of those values. This can be programmed in Python with this simple line:

In [ ]:
((abs(prices - prices.rolling(50).mean())) / prices).mean()

Even though this is a different metric than the high/low metric we previously calculated, it shows the same result: The Russell 2000 has been more volatile than the S&P 500 over the last 10 years.

In the first video of this lesson, we will recap the risk metrics we've written thus far and show how we can use them in Python.

In [ ]:
VimeoVideo("706651510", h="e6511ea8b5", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1a48fxAGWt2gibIXJ7VJD_u2ASuBWIEeC/view?usp=sharing)

### 2.3 Price Volatility: Standard Deviations
Standard deviation is potentially the most popular volatility metric used when looking at an investment opportunity. It’s as simple as calling the std() method on a DataFrame.

In [ ]:
prices.std()

As mentioned before, calling this on the prices is not as intuitive as calling this on the returns since prices start at different points. For a better comparison, we will call this std() method on the daily returns of the last 10 years:

In [ ]:
df.std()

All three of our volatility metrics, albeit slightly similar, show us that the Russell 2000 Index has been more volatile over the last 10 years than the S&P 500.



### 2.4 Writing a Comparison Function in Python
To tie it all together, we're going to take the three volatility metrics we've used up to this point, along with average daily return, and write a function, which takes:

* `startTime` - `dateTime` format
* `endTime` - `dateTime` format
* `tickers` - a dictionary of values where the key is the yahoo ticker and the value is the display name i.e. {"^GSPC": "SP500", "^RUT": "Russell2000"}

By writing a function like this, it allows our research to be reproducible and applicable to many different parameters. In our case, it will be a date range and dictionary of tickers. If you were writing a serious application, you would need to have lots of error handling in this function, such as guaranteeing that parameters passed in are of the correct type. In our case, we will assume the data types of parameters are correct and that the date range is longer than 365 days. 

In [ ]:
def investCompare(startTime, endTime, tickers):
    # pull price data from yahoo -- (list(tickers.keys())) = ['^GSPC','^RUT']
    prices = web.DataReader(list(tickers.keys()), "yahoo", startTime, endTime)[
        "Adj Close"
    ]
    prices = prices.rename(columns=tickers)
    returns = np.log(prices) - np.log(prices.shift(1))
    returns = returns.iloc[1:, 0:]

    # pull data into separate DataFrame, 52weeks to just look at the last 365 days of data for calculating our high/low metric
    currYear = prices.loc[
        date.today() - datetime.timedelta(365) : date.today()  # noqa E203
    ]
    highLow = (currYear.max() - currYear.min()) / prices.iloc[-1]
    highLow = pd.DataFrame(highLow, columns=["HighMinusLow"])

    # Moving average volatility
    MA = pd.DataFrame(
        ((abs(prices - prices.rolling(50).mean())) / prices).mean(),
        columns=["MovingAverageVolatility"],
    )

    investments = pd.merge(highLow, MA, on="Symbols")
    investments = pd.merge(
        investments,
        pd.DataFrame(returns.std(), columns=["StandardDeviation"]),
        on="Symbols",
    )
    investments = pd.merge(
        investments,
        pd.DataFrame(100 * returns.mean(), columns=["Daily Return Percentage"]),
        on="Symbols",
    )

    return investments.round(3)

### 2.5 Trying It Out
We will start by calling our function with the two indices we've been examining thus far: the S&P 500 alongside the Russell 2000 Index. As we've seen, the three volatility metrics showed the Russell 2000 to be the more volatile investment. The average daily return is also higher for the S&P 500. Therefore, over the last 10 years not only has the S&P 500 returned better than the Russell 2000, but it was also less volatile. Keep in mind that these different volatility metrics all calculate in some way how turbulently the stock has behaved in the past. This exercise was intended to demonstrate different ways of calculating volatility and how you can be a bit creative with it. This doesn't mean that volatility is constant or easily predictable, behaving the same way going forward; this exercise only reflects the past.

In [ ]:
investCompare(
    datetime.date(2020, 1, 1),
    datetime.date.today(),
    {"^GSPC": "SP500", "^RUT": "Russell2000"},
)

This function gives us one clean space to compare investments in one line.

We also want to show that we made the function flexible in order to add symbols in the future. In the example below, we added Apple to the DataFrame in order to compare it to the other two indices. 

In [ ]:
investCompare(
    datetime.date(2020, 1, 1),
    datetime.date.today(),
    {"^GSPC": "SP500", "^RUT": "Russell2000", "AAPL": "Apple"},
)

These metrics show Apple to be a better investment in terms of daily return, but they also show that Apple is significantly more volatile than the two indices. This is expected, since the indices represent vast baskets of stock. With many stocks moving in different directions, you'd expect the indices to be less volatile than an individual stock, even a blue-chip stock like Apple.

In the next video, we will combine all of the risk metrics we've created so far and wrap them up tidily into a function that can be reused going forward.

In [ ]:
VimeoVideo("706651724", h="836ae485f0", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1na-nRzhNunGctPmXHUKAwAuASjxF3d3x/view?usp=sharing)

## 2.6 Comparing Growth vs. Value stocks
We can continue using the function we just wrote to examine and compare different categories of investments, starting here with growth vs. value stocks. As a proxy for growth stocks, we will use the Vanguard Growth ETF (VUG). VUG is one of the biggest growth ETFs with around \\$87 billion in assets under management. For value stocks, we will use the Vanguard Value ETF (VTV). VTV is the most prominent value ETF with \\$88 billion in assets under management. As you may imagine by now, Vanguard is a popular purveyor of ETFs and has some of the lowest expense ratios in the industry.

In [ ]:
investCompare(
    datetime.date(2020, 1, 1), datetime.date.today(), {"VUG": "Growth", "VTV": "Value"}
)

Running `investCompare` over the last two years for these ETFs shows that growth stocks have clearly outperformed value stocks in terms of returns—but not without increased volatility. We need to keep in mind that time frames have a large influence on our results here. Let's take a look at the data since 2010:

In [ ]:
investCompare(
    datetime.date(2010, 1, 1), datetime.date.today(), {"VUG": "Growth", "VTV": "Value"}
)

We do see quite similar results here, with the daily returns of the growth ETF coming down to earth a bit. Growth stocks tend to be more volatile in general, and the proof can be seen empirically here.

## 2.7 Comparing Domestic vs. Foreign Stocks
We will use the S&P 500 once again for U.S. stocks while finding an ETF for the foreign stocks. There are tons of options to choose from in this space. 

The first question you need to ask yourself is where do you want to invest? It could be emerging markets, Europe, China, etc. To make things simple, we will use one example that is specific to Europe, ETF: SPDR Portfolio Europe ETF (SPEU), and one example that is specific to China, ETF: SPDR S&P China ETF (GXC).

In [ ]:
investCompare(
    datetime.date(2020, 1, 1),
    datetime.date.today(),
    {"^GSPC": "SP500", "SPEU": "Europe ETF", "GXC": "China ETF"},
)

Since the start of 2020, U.S. stocks have had the best daily returns by a solid margin with lower volatility than Europe and China. Hindsight is 20/20, but this makes a clear case for U.S. equities being the best investment over the last two years. If we zoom out and look at the data since 2010, we'll see the following:

In [ ]:
investCompare(
    datetime.date(2010, 1, 1),
    datetime.date.today(),
    {"^GSPC": "SP500", "SPEU": "Europe ETF", "GXC": "China ETF"},
)

The above makes an even better case for U.S. equities over the last 10 years. That being said, this is no guarantee that the next 10 years will look the same. Past performance is no guarantee of future results.

## 3. Conclusion

In this lesson, we compared returns of different investment opportunities, and we used Python to illustrate different ways of calculating the volatility of investments. Keep these lessons in mind as we delve deeper into the connection between volatility and return in the next lesson. Up next, we'll expand this analysis by also taking into account the returns of different investments.

**References**

- Standard & Poor's. *S&P U.S. Indices Methodology*, Standard & Poor's, Nov. 2021, https://www.spglobal.com/spdji/en/documents/methodologies/methodology-sp-us-indices.pdf

- Maginn, John L. *Managing Investment Portfolios Workbook: A Dynamic Process*. John Wiley & Sons, Inc., 2007.

---
Copyright © 2022 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
